# **0. Install Depedencies**

In [1]:
!pip install tflite_model_maker

     |████████████████████████████████| 593kB 2.9MB/s 
     |████████████████████████████████| 645kB 12.9MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 6.3MB 16.5MB/s 
     |████████████████████████████████| 686kB 27.3MB/s 
     |████████████████████████████████| 1.2MB 37.5MB/s 
     |████████████████████████████████| 112kB 25.1MB/s 
     |████████████████████████████████| 122kB 38.2MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 1.1MB 32.9MB/s 
     |████████████████████████████████| 849kB 33.6MB/s 
     |████████████████████████████████| 174kB 35.1MB/s 
     |████████████████████████████████| 194kB 32.7MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 358kB 31.9MB/s 
     |████████████████████████████████| 38.2MB 75kB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=dd31a960f20a5b30d2dc6ca0

# **1. Import Libraries**


In [2]:
#Import libraries
#from transformers import DistilBertTokenizerFast
#from transformers import TFDistilBertForSequenceClassification

import tensorflow as tf

In [3]:
#Some libraries from APLOD-Net V1.0

import pandas as pd
import numpy as np

# **2. EDA (Exploratory Data Analysis)**

In [5]:
df = pd.read_csv('train.csv')
df.head()

,sentimen,Tweet
0,0.0,barusan liat tulisan di belakang truk rela inj...
1,0.0,her itu lho miss kevin sama keven rebutan gimb...
2,0.0,iya rep gatau aku masih kelas 4 sd ehh di block
3,0.0,aku mohon tepatilah janjimu penantiancintaeps19
4,0.0,bukan beria nk kahwin sbb gatal celah kangkang...


In [6]:
labels = []

for index,row in df.iterrows():
  if row['sentimen'] == (6.0) :
    labels.append(6)
  elif row['sentimen'] == (5.0) :
    labels.append(5)
  elif row['sentimen'] == (4.0) :
    labels.append(4)
  elif row['sentimen'] == (3.0) :
    labels.append(3)
  elif row['sentimen'] == (2.0) :
    labels.append(2)
  elif row['sentimen'] == (1.0) :
    labels.append(1)
  else :
    labels.append(0)

df['labels'] = labels
df = df.drop(columns='sentimen')
df.head()

,Tweet,labels
0,barusan liat tulisan di belakang truk rela inj...,0
1,her itu lho miss kevin sama keven rebutan gimb...,0
2,iya rep gatau aku masih kelas 4 sd ehh di block,0
3,aku mohon tepatilah janjimu penantiancintaeps19,0
4,bukan beria nk kahwin sbb gatal celah kangkang...,0


In [7]:
df['labels'].value_counts()

0    5328
2    4828
1    2792
4     152
6     100
3      62
5      29
Name: labels, dtype: int64

In [8]:
df_train_preprocessed = df.copy()
df_train_preprocessed

,Tweet,labels
0,barusan liat tulisan di belakang truk rela inj...,0
1,her itu lho miss kevin sama keven rebutan gimb...,0
2,iya rep gatau aku masih kelas 4 sd ehh di block,0
3,aku mohon tepatilah janjimu penantiancintaeps19,0
4,bukan beria nk kahwin sbb gatal celah kangkang...,0
...,...,...
13286,"Jangan dibahas dong, udah paham aja kan di kau...",6
13287,di agama lo nyembah nya siapa sih? masa Tuhan ...,6
13288,"Katanya demi kemanusiaan, kemanusiaan milik ma...",6
13289,Orang timur kurang pintar dari orang barat,6


In [9]:
#Function untuk Cleansing Data
import string, re

def cleansing(data):
  #lower text
  data = data.lower()

  #hapus tanda baca
  remove = string.punctuation
  translator = str.maketrans(remove, ' '*len(remove))
  data = data.translate(translator)

  #hapus ASCII dan UNICODE
  data = data.encode('ascii', 'ignore').decode('utf-8')
  data = re.sub(r'[^\x00-\x7f]','r',data)

  #remove newline
  data = data.replace('\n', ' ')

  return data

In [10]:
#Aplikasikan Function cleansing ke dataset kita
sentences = []

for index,row in df_train_preprocessed.iterrows():
  sentences.append(cleansing(row['Tweet']))

df_train_preprocessed['Tweet'] = sentences
df_train_preprocessed.head()

,Tweet,labels
0,barusan liat tulisan di belakang truk rela inj...,0
1,her itu lho miss kevin sama keven rebutan gimb...,0
2,iya rep gatau aku masih kelas 4 sd ehh di block,0
3,aku mohon tepatilah janjimu penantiancintaeps19,0
4,bukan beria nk kahwin sbb gatal celah kangkang...,0


In [11]:
#install library Sastrawi
!pip install Sastrawi

     |████████████████████████████████| 215kB 3.0MB/s 


In [12]:
#import library
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

#contoh
kalimat = 'saya mencoba dengan library sastrawi pada project kali ini'
stop = stopword.remove(kalimat)
print(stop)

mencoba library sastrawi project kali


In [13]:
#aplikasikan pada dataset kita
sentences = []

for index,row in df_train_preprocessed.iterrows():
  sentences.append(stopword.remove(row['Tweet']))

df_train_preprocessed['Tweet'] = sentences
df_train_preprocessed.head()

,Tweet,labels
0,barusan liat tulisan belakang truk rela injek ...,0
1,her lho miss kevin sama keven rebutan gimbot w...,0
2,iya rep gatau aku kelas 4 sd ehh block,0
3,aku mohon tepatilah janjimu penantiancintaeps19,0
4,bukan beria nk kahwin sbb gatal celah kangkang...,0


In [14]:
#Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#contoh
kalimat = 'barangnya bagus banget kak, aku suka. pengirimannya juga cepet'
katadasar = stemmer.stem(kalimat)

print(katadasar)

barang bagus banget kak aku suka kirim juga cepet


In [15]:
#aplikasikan ke dataset kita

sentences = []
for index,row in df_train_preprocessed.iterrows():
  sentences.append(stemmer.stem(row['Tweet']))

df_train_preprocessed['Tweet'] = sentences
df_train_preprocessed.head()

,Tweet,labels
0,barusan liat tulis belakang truk rela injek ko...,0
1,her lho miss kevin sama keven rebut gimbot wis...,0
2,iya rep gatau aku kelas 4 sd ehh block,0
3,aku mohon tepat janji penantiancintaeps19,0
4,bukan ria nk kahwin sbb gatal celah kangkang t...,0


In [16]:
df_train_preprocessed.shape

(13291, 2)

In [17]:
df_train_preprocessed.to_csv (r'dataframe.csv', index = False, header=True)

# **3. Setup Mobile-BERT and Train the Model**

In [18]:
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

In [20]:
mb_spec = model_spec.get('mobilebert_classifier')

In [21]:
mb_train_data = DataLoader.from_csv(
      filename='dataframe.csv',
      text_column='Tweet',
      label_column='labels',
      model_spec=mb_spec,
      is_training=True)

In [23]:
train_data , test_data = mb_train_data.split(0.8)

In [25]:
mb_model = text_classifier.create(train_data, model_spec=mb_spec, epochs=3, batch_size =  8, shuffle=False)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/3
1329/1329 [==============================] - 442s 289ms/step - loss: 0.5115 - test_accuracy: 0.9498
Epoch 2/3
1329/1329 [==============================] - 377s 284ms/step - loss: 0.3702 - test_accuracy: 0.9598
Epoch 3/3
1329/1329 [==============================] - 377s 284ms/step - loss: 0.5113 - test_accuracy: 0.9425


In [26]:
loss , acc = mb_model.evaluate(test_data)

84/84 [==============================] - 25s 228ms/step - loss: 0.3675 - test_accuracy: 0.9507


# **4.Save the model**

In [27]:
mb_model.export(export_dir='mobilebert/')

INFO:tensorflow:Assets written to: /tmp/tmp15yo_579/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp15yo_579/saved_model/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpa404p01v/vocab.txt.


...

INFO:tensorflow:The associated file that has been been packed to the model is:


INFO:tensorflow:['vocab.txt', 'labels.txt']


INFO:tensorflow:['vocab.txt', 'labels.txt']


INFO:tensorflow:TensorFlow Lite model exported successfully: mobilebert/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: mobilebert/model.tflite


In [28]:
mb_model.export(export_dir='/content/mobilebert/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB, ExportFormat.TFLITE, ExportFormat.SAVED_MODEL])

INFO:tensorflow:Assets written to: /tmp/tmpq09jb0mh/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpq09jb0mh/saved_model/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmp1pnf5ezi/vocab.txt.


...

INFO:tensorflow:Assets written to: /content/mobilebert/saved_model/assets


INFO:tensorflow:Saved vocabulary in /content/mobilebert/vocab.txt.


INFO:tensorflow:Saved vocabulary in /content/mobilebert/vocab.txt.


INFO:tensorflow:Saving labels in /content/mobilebert/labels.txt


INFO:tensorflow:Saving labels in /content/mobilebert/labels.txt


In [30]:
accuracy = mb_model.evaluate_tflite('/content/mobilebert/model.tflite', test_data)
print('TFLite model accuracy: ', accuracy)

TFLite model accuracy:  {'accuracy': 0.9507333584054156}


In [34]:
!pip freeze > requirements.txt

In [35]:
from google.colab import files
files.download("/content/mobilebert")
files.download("train.csv")
files.download("dataframe.csv")
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>